In [4]:
import os
from PIL import Image
import numpy as np
import cv2
import pickle

BASE_DIR = os.path.dirname(os.path.abspath("PycharmProject"))
image_dir = os.path.join(BASE_DIR, "Images")

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

current_id = 0
label_ids = {}
x_train = []
y_labels = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jfif"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
#             print(label,path)
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
#             print(label_ids)
            
            pil_image = Image.open(path).convert("L")
            image_array = np.array(pil_image, "uint8")
#             print(image_array)
            
            faces = face_cascade.detectMultiScale(image_array,1.5,5)
            
            for (x,y,w,h) in faces:
                region_of_interest = image_array[y:y+h, x:x+w]
                x_train.append(region_of_interest)
                y_labels.append(id_)
                
with open("labels.pickle", "wb") as f:
    pickle.dump(label_ids, f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")


AttributeError: module 'cv2.cv2' has no attribute 'face'